<a href="https://colab.research.google.com/github/asdswsxs/colab/blob/main/signate_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, gc, sys
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
from simpletransformers.classification import ClassificationModel
import torch

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
SEED = 512
BASE_PATH = 'signate/'
TEXT_COL = "description"
TARGET = "jobflag"
NUM_CLASS = 4
N_FOLDS = 4

In [ ]:
print(torch.__version__)

1.6.0+cu101


In [ ]:
def metric_f1(labels, preds):
    return f1_score(labels, preds, average='macro')

In [ ]:
train = pd.read_csv(BASE_PATH+"train.csv").drop(['id'], axis=1)
train = train.rename(columns={TEXT_COL:'text', TARGET:'label'})
train['label'] -= 1

test = pd.read_csv(BASE_PATH+"test.csv")
test = test.rename(columns={TEXT_COL:'text'}).drop(['id'], axis=1)

In [ ]:
"""
import re 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
import nltk

nltk.download("stopwords")
#list_stopwords = list(set(stopwords.words("english")))
#combined = train.append(test, ignore_index=True, sort=True)
stemmer = PorterStemmer()

def cleaning(texts):
    clean_texts = []
    for text in texts:
        #アルファベット以外をスペースに置き換え
        clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        #単語長が3文字以下のものは削除する
        clean_short_tokenized = [word for word in clean_punc.split() if len(word) > 3]
        #ステミング
        clean_normalize = [stemmer.stem(word) for word in clean_short_tokenized]
        #単語同士をスペースでつなぎ, 文章に戻す
        clean_text = ' '.join(clean_normalize)
        clean_texts.append(clean_text)
    return clean_texts

train_cleaned = train.copy()
train['text'] = cleaning(train['text'])
test["text"] = cleaning(test["text"])
#print('#cleaned\n', train['text'][0])
print(test["text"][0])
"""

'\nimport re \nfrom nltk.stem.porter import PorterStemmer\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import sent_tokenize,word_tokenize\nimport nltk\n\nnltk.download("stopwords")\n#list_stopwords = list(set(stopwords.words("english")))\n#combined = train.append(test, ignore_index=True, sort=True)\nstemmer = PorterStemmer()\n\ndef cleaning(texts):\n    clean_texts = []\n    for text in texts:\n        #アルファベット以外をスペースに置き換え\n        clean_punc = re.sub(r\'[^a-zA-Z]\', \' \', text)\n        #単語長が3文字以下のものは削除する\n        clean_short_tokenized = [word for word in clean_punc.split() if len(word) > 3]\n        #ステミング\n        clean_normalize = [stemmer.stem(word) for word in clean_short_tokenized]\n        #単語同士をスペースでつなぎ, 文章に戻す\n        clean_text = \' \'.join(clean_normalize)\n        clean_texts.append(clean_text)\n    return clean_texts\n\ntrain_cleaned = train.copy()\ntrain[\'text\'] = cleaning(train[\'text\'])\ntest["text"] = cleaning(test["text"])\n#print(\'#cleaned\n\', train[

In [ ]:
#交叉验证，分割train数据集进行测试

kfold = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
train['fold_id'] = -1
for fold, (train_idx, valid_idx) in enumerate(kfold.split(train.index, train['label'])):
    train.loc[train.iloc[valid_idx].index, 'fold_id'] = fold

X_train = train.loc[train['fold_id']!=0]
X_valid = train.loc[train['fold_id']==0]

In [ ]:
!pip install utils

In [ ]:
import wandb
#from utils import load_rte_data_file
from statistics import mean, mode
import logging

sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3,7,10,12]},
        "learning_rate": {"min": 5e-5, "max": 1e-4},
    },
}
sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Create sweep with ID: fhs4zmk5
Sweep URL: https://app.wandb.ai/asdswsxs/Simple%20Sweep/sweeps/fhs4zmk5


In [ ]:
params = {
    "output_dir": "outputs/",
    "max_seq_length": 128,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    "manual_seed":SEED,
    "overwrite_output_dir":True,
    "save_model_every_epoch":False,
    "use_early_stopping":True, 
}
Robert_model = ClassificationModel("roberta","roberta-base",use_cuda=True,num_labels=4,args=params,weight=[0.2314, 0.1833, 0.1982, 0.3872])
Robert_model.lazy_loading = True

- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import wandb
#from utils import load_rte_data_file
from statistics import mean, mode
import logging

sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [7,10,12]},
        "learning_rate": {"min": 5e-5, "max": 1e-4},
    },
}
sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

def train():
    # Initialize a new wandb run
    wandb.init()
    torch.multiprocessing.set_start_method('spawn')
    # Create a TransformerModel
    params = {
        "output_dir": "outputs/",
        "max_seq_length": 128,
        "train_batch_size": 128,
        "eval_batch_size": 64,
        "manual_seed":SEED,
        "overwrite_output_dir":True,
        "save_model_every_epoch":False,
        "use_early_stopping":True, 
        "wandb_project":"Simple Sweep",
        "reprocess_input_data ":True,
        "evaluate_during_training ":True,

    }
    Robert_model = ClassificationModel("roberta","roberta-base",use_cuda=True,num_labels=4,args=params,weight=[0.2314, 0.1833, 0.1982, 0.3872])

    # Train the model
    Robert_model.train_model(train_df, eval_df=eval_df)

    # Evaluate the model
    Robert_model.eval_model(eval_df)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)


Create sweep with ID: fgdqnt6h
Sweep URL: https://app.wandb.ai/asdswsxs/Simple%20Sweep/sweeps/fgdqnt6h


INFO:wandb.wandb_agent:Running runs: []
INFO:wandb.wandb_agent:Agent received command: run
INFO:wandb.wandb_agent:Agent starting run with config:
	learning_rate: 9.498857399626201e-05
	num_train_epochs: 10


wandb: Agent Starting Run: f9m3zvua with config:
	learning_rate: 9.498857399626201e-05
	num_train_epochs: 10
wandb: Agent Started Run: f9m3zvua


INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOmY5bTN6dnVhOlNpbXBsZSBTd2VlcDphc2Rzd3N4cw==
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving patches
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers
Process Process-9:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/wandb/wandb_agent.py", line 64, in _start
    function()
  File "<ipython-input-25-836fe6a80c2c>", line 22, in train
    torch.multi

In [ ]:
Robert_model.train_model(X_train,out_dir="cache_dir")

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:282: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


In [ ]:
result2, Robert_model_outputs, wrong_predictions = Robert_model.eval_model(X_valid, f1=metric_f1)
print(result2)
y_pred, raw_outputs = Robert_model.predict(test['text'])
print(y_pred)

#y_pred, raw_outputs = bert_model.predict(test['text'])
#print(y_pred)

test = pd.read_csv(BASE_PATH+"test.csv")
submit = pd.DataFrame({'index':test['id'], 'pred':y_pred+1})
submit

submit.to_csv(BASE_PATH+"submit_Robert_model6.0_bert.csv", index=False, header=False)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:754: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.5096973214169482, 'f1': 0.6135243196755684, 'eval_loss': 1.091469665368398}



[3 2 2 ... 0 2 2]
